In [6]:
import json
import ast
import numpy as np
import sklearn
import subprocess

# Data Wrangling

In [20]:
DATA_PATH = '/home/linghao/Question-Relevance-in-VQA/data/true_vs_false_premise/simple_models/'
DIM_TXT_VEC = 100

In [4]:
data = json.load(open(DATA_PATH+'data_transformed.json'))

In [9]:
questions = []
for t in data:
    questions += t['question']+'\n'
with open(DATA_PATH+'questions.raw', 'w') as f:
    f.writelines(questions)

In [19]:
cmd = '/home/linghao/fastText-0.1.0/fasttext skipgram -input %s -output model' % (DATA_PATH+'questions.raw')
subprocess.call(cmd, shell=True)

0

In [18]:
cmd = '/home/linghao/fastText-0.1.0/fasttext print-sentence-vectors model.bin < %s > %s' % \
        (DATA_PATH+'questions.raw', DATA_PATH+'questions.vec')
subprocess.call(cmd, shell=True)

0

In [6]:
f = open('questions.vec')

In [7]:
features = []
labels = []
i = 0
for line in f:
    img_vec = ast.literal_eval(data[i]['image_features_fc7'])
    txt_vec = [float(x) for x in line.strip().split(' ')[-DIM_TXT_VEC:]]
    feat_vec = img_vec + txt_vec
    features.append(feat_vec)
    labels.append(data[i]['relevance'])
    i += 1

In [11]:
features = np.array(features)
labels = np.array(labels)

In [10]:
features.shape

(107822, 400)

In [12]:
labels.shape

(107822,)

# Model Training

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=10805)

## Logistic Regression

In [23]:
lr = LogisticRegression()

In [24]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
y_pred = lr.predict(X_test)

In [36]:
print('Accuracy: %.4f' % (sum(y_pred == y_test) / len(y_test)))

Accuracy: 0.6100


In [37]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      False       0.60      0.64      0.62     16159
       True       0.62      0.58      0.60     16188

avg / total       0.61      0.61      0.61     32347



## Random Forests

In [14]:
rfc = RandomForestClassifier(n_estimators=100)

In [17]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
y_pred = rfc.predict(X_test)

In [34]:
print('Accuracy: %.4f' % (sum(y_pred == y_test) / len(y_test)))

Accuracy: 0.7217


In [32]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      False       0.75      0.67      0.71     16159
       True       0.70      0.77      0.74     16188

avg / total       0.72      0.72      0.72     32347

